In [1]:
import requests 
import json
import pandas as pd 

league_id = "LEAGUE_ID";
username = "slex@udel.edu";
password = "packer123";
year = "2022";


In [2]:
league_id = '62106'
username = 'slex@udel.edu'
password = 'packer123'
year = '2022'
login_url = f'https://api.myfantasyleague.com/{year}/login?USERNAME={username}&PASSWORD={password}&XML=1'
login_req = requests.get(login_url)

In [3]:
players_req = 'https://www47.myfantasyleague.com/2022/export?TYPE=players&L=62106&APIKEY=&DETAILS=&SINCE=&PLAYERS=&JSON=1'
s = requests.Session()
s.post(login_url)
players = s.get(players_req)
player_keys = players.content
player_string = player_keys.decode('utf-8') 
j_player = json.loads(player_string)
player_ids = pd.json_normalize(j_player['players'], record_path = ['player'])

In [5]:
franchise_ids = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010']

franchise_dict = {'0001' : 'Whistleblowers', 
                  '0002' : 'Toney Carr and Van Repair',
                  '0003' : 'Pass The Herb',
                  '0004' : 'Burksy Boot Camp',
                  '0005' : 'Jojo Juju',
                  '0006' : 'Plant Based',
                  '0007' : 'TeamKeam',
                  '0008' : 'Show Me The Mooney',
                  '0009' : 'Nick Six',
                  '0010' : 'Hurts So Good',
                  }
                  
main_rosters = pd.DataFrame()
for franchise in franchise_ids : 
    rosters_req = f'https://api.myfantasyleague.com/2022/export?TYPE=rosters&L=62106&APIKEY=&FRANCHISE={franchise}&W=&JSON=1'
    s = requests.Session()
    s.post(login_url)
    rosters = s.get(rosters_req)
    players = s.get(players_req)
    roster_data = rosters.content
    roster_string = roster_data.decode('utf-8') 
    j_roster = json.loads(roster_string)
    roster_df = pd.json_normalize(j_roster['rosters']['franchise'], record_path = ['player'])
    roster_merge = roster_df.merge(player_ids, how='left', on='id')
    roster_merge.rename(columns = {'status_x' : 'roster_spot', 'status_y' : 'rookie'}, inplace=True)
    roster_merge['franchise'] = franchise_dict[franchise]
    roster_merge.rookie.fillna('', inplace=True)
    main_rosters = main_rosters.append(roster_merge)

In [6]:
#main_rosters.loc[main_rosters['franchise']=='Whistleblowers', :]
salary_map = {'ROSTER' : 1.0, 
              'IR'     : .5, 
              'TAXI_SQUAD' : .25}

main_rosters['salary_scaler'] = main_rosters['roster_spot'].map(salary_map)
main_rosters['salary'].astype(float)
main_rosters['effective_salary'] = main_rosters['salary'].astype(float)*main_rosters['salary_scaler']
main_rosters.groupby('franchise', as_index=False)['effective_salary'].sum()

,franchise,effective_salary
0,Burksy Boot Camp,277.0500
1,Hurts So Good,223.5050
2,Jojo Juju,408.0425
3,Nick Six,399.4100
4,Pass The Herb,316.7325
5,Plant Based,371.6125
6,Show Me The Mooney,336.1975
7,TeamKeam,322.9500
8,Toney Carr and Van Repair,218.1175
9,Whistleblowers,384.7825


In [10]:


picks_req = 'https://www47.myfantasyleague.com/2022/export?TYPE=futureDraftPicks&L=62106&APIKEY=&JSON=1'
s = requests.Session()
s.post(login_url)
picks = s.get(picks_req)
picks_data = picks.content
picks_string = picks_data.decode('utf-8') 
j_picks = json.loads(picks_string)

master_picks = pd.DataFrame() 
for i in range(0, 10) : 
    picks_df = pd.json_normalize(j_picks['futureDraftPicks']['franchise'][i], record_path =['futureDraftPick'])
    picks_df['franchise'] = franchise_dict[franchise_ids[i]]
    picks_df['original_franchise'] = picks_df['originalPickFor'].map(franchise_dict)
    picks_df.drop('originalPickFor', axis='columns', inplace=True)
    master_picks = master_picks.append(picks_df[['franchise', 'year', 'round', 'original_franchise']])
master_picks

,franchise,year,round,original_franchise
0,Whistleblowers,2023,1,Whistleblowers
1,Whistleblowers,2023,2,Whistleblowers
2,Whistleblowers,2023,3,Whistleblowers
3,Whistleblowers,2023,4,Whistleblowers
4,Whistleblowers,2023,5,Whistleblowers
...,...,...,...,...
13,Hurts So Good,2025,3,Hurts So Good
14,Hurts So Good,2025,4,Hurts So Good
15,Hurts So Good,2025,5,Hurts So Good
16,Hurts So Good,2025,6,Hurts So Good


In [13]:
#Pull in trade valuations 
file_location = 'C:\\NFL\\player_values.csv'

values = pd.read_csv(file_location)
values.to_csv('player_values.csv')